In [683]:
# selenium 라이브러리에서 webdriver 모듈 임포트, 웹 브라우저를 자동으로 제어하기 위한 기능을 제공
from selenium import webdriver
# 크롬 웹드라이버를 위한 서비스 객체를 사용하기 위해 Service 모듈 임포트
from selenium.webdriver.chrome.service import Service
# 크롬 드라이버의 자동 업데이트를 위해 webdriver_manager에서 ChromeDriverManager를 임포트
from webdriver_manager.chrome import ChromeDriverManager
# 크롬 옵션을 설정하기 위한 Options 모듈을 임포트 일반적으로 브라우저의 특정 옵션을 설정할 때 사용
from selenium.webdriver.chrome.options import Options
# html요소 접근 전략
from selenium.webdriver.common.by import By
# 키보드 조작
from selenium.webdriver.common.keys import Keys

import pandas as pd
from bs4 import BeautifulSoup
import pyautogui
import time
import re

# 크롬 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach",True)
# 불필요한 에레 메세지 제거
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

In [ ]:
# ChromeDriverManager를 사용해 버전에 맞는 웹드라이버를 다운로드하여 해당 경로를 셀레니움에 전달
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# 웹페이지가 로딩될대까지 2초대기
driver.implicitly_wait(2)
# 브라우저 크기 최대화
driver.maximize_window()

In [ ]:
url = 'https://map.naver.com'
driver.get(url)
time.sleep(2)

In [ ]:
# 검색어 입력
input_search = driver.find_element(By.CSS_SELECTOR,'input.input_search')
input_search.click()
time.sleep(0.5)
input_search.clear()
input_search.send_keys('울산 북구 맛집')
time.sleep(0.5)
input_search.send_keys(Keys.ENTER)
time.sleep(1)

In [ ]:
#필터링 (많이 찾는)

# switch to selected iframe
driver.switch_to.frame('searchIframe')
driver.find_element(By.CSS_SELECTOR, '#app-root .T46Lb.pTP9T.LXtsf').click()
time.sleep(0.2)

# 많이 찾는 클릭!
driver.find_element(By.CSS_SELECTOR, '.qP6uz.SbSKx').click()
time.sleep(0.2)

# 결과보기 버튼 클릭
driver.find_element(By.CSS_SELECTOR, '#modal-root .GZe5x.Mj_l3').click()
time.sleep(0.5)


In [ ]:
# 무한 스크롤
driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container').click()

# 로딩된 데이터 개수 확인
lis = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS.rTjJo')
prev_len = len(lis)
print(prev_len)
while True :
    # 맨 아래로 스크롤 내림
    driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.END)    
    time.sleep(0.5)
    
    # 스크롤 내린후 li 갯수
    lis = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS.rTjJo')
    curr_len = len(lis)
    
    print(f'스크롤전:{prev_len}, 스크롤후:{curr_len}')
    if curr_len == prev_len :
        break
    prev_len = curr_len        

In [ ]:
# '광고'라는 단어를 포함하지 않는 'li' 요소들의 리스트를 생성합니다.
# lis = [li for li in lis if '광고' not in li.text and '제주옥탑 울산송정점' in li.text]
lis = [li for li in lis if '광고' not in li.text]

In [703]:
def get_reviews_of_stores (stores, moreview=1):
    """
        stores: 상점 목록
        moreview : 더보기 횟수
    """
    
    review_list = []

    # 광고 제외 하고 리뷰에 접근    
    for li in stores:

        # 1. 사업장 클릭    
        li.find_element(By.CSS_SELECTOR, 'div.CHC5F > a.tzwk0').click()
        time.sleep(1)

        # searchIframe 탈출
        driver.switch_to.default_content()

        # entryIframe 진입   
        driver.switch_to.frame('entryIframe')

        # 사업장명
        business_name = driver.find_element(By.CSS_SELECTOR,'#_title > div > span.Fc1rA').text
        # 업태명
        business_type = driver.find_element(By.CSS_SELECTOR,'#_title > div > span.DJJvD').text

        # 방문자리뷰건수        
        # 'span.PXMot' 요소를 찾습니다.
        visitor_reviews = driver.find_element(By.CSS_SELECTOR, 'span.PXMot')
        # 해당 요소 내부에서 'a' 태그를 모두 찾습니다.
        links = visitor_reviews.find_elements(By.TAG_NAME, 'a')

        # 'a' 태그의 존재 여부를 체크합니다.
        if not links:
            visitor_reviews = driver.find_element(By.CSS_SELECTOR, 'span.PXMot + span.PXMot')

        pattern = r"\d{1,3}(?:,\d{3})*"
        cnt_of_vr = re.search(pattern,visitor_reviews.text).group()   

        # 방문자리뷰 클릭
        visitor_reviews.click()
        time.sleep(0.5)

        # 리뷰 가져오기
        cnt = 0
        while True:

            # 더보기 횟수 제한
            if cnt == moreview:
                break   

            # 맨 아래로 스크롤 내림
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.2)       

            # 리뷰펼치기
            reviews_lis = driver.find_elements(By.CSS_SELECTOR, 'div.place_section.k1QQ5 div.place_section_content > ul li')
            for reviews_li in reviews_lis:
                driver.execute_script('arguments[0].click();', reviews_li.find_element(By.CSS_SELECTOR,'.vg7Fp a'))
                time.sleep(0.2)

            # 리뷰 스크랩
            for reviews_li in reviews_lis:
                review_date = driver.execute_script('return arguments[0].textContent',reviews_li.find_element(By.CSS_SELECTOR,'.jxc2b .D40bm .CKUdu .place_blind:nth-of-type(2)'))           
                review_text = driver.execute_script('return arguments[0].textContent',reviews_li.find_element(By.CSS_SELECTOR,'.vg7Fp a'))

                tmp_reivew = {
                    'business_name':business_name,
                    'business_type':business_type,
                    'cnt_of_vr':cnt_of_vr,
                    'review_date':review_date,
                    'review_text':review_text
                }
                review_list.append(tmp_reivew)            

            try:
                # 더보기 클릭
                driver.find_element(By.CSS_SELECTOR, 'a.fvwqf').click()
                time.sleep(0.2)

            except Exception:
                # 더보기 없으면 멈추기
                print("요소가 존재하지 않습니다.")
                break


            cnt += 1

        driver.switch_to.default_content()    
        # searchIframe 진입
        driver.switch_to.frame('searchIframe') 
                    
    return pd.DataFrame(review_list)

In [705]:
# ChromeDriverManager를 사용해 버전에 맞는 웹드라이버를 다운로드하여 해당 경로를 셀레니움에 전달
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# 웹페이지가 로딩될대까지 2초대기
driver.implicitly_wait(2)
# 브라우저 크기 최대화
driver.maximize_window()

keywords = ['울산 북구 맛집','울산 동구 맛집','울산 남구 맛집','울산 중구 맛집','울산 울주군 맛집',]

for keyword in keywords:
    url = 'https://map.naver.com'
    driver.get(url)
    time.sleep(2)

    # 검색어 입력
    input_search = driver.find_element(By.CSS_SELECTOR,'input.input_search')
    input_search.click()
    time.sleep(0.5)
    input_search.clear()
    input_search.send_keys(keyword)
    time.sleep(0.5)
    input_search.send_keys(Keys.ENTER)
    time.sleep(1)

    #필터링 (많이 찾는)
    # switch to selected iframe
    driver.switch_to.frame('searchIframe')
    driver.find_element(By.CSS_SELECTOR, '#app-root .T46Lb.pTP9T.LXtsf').click()
    time.sleep(0.2)

    # 많이 찾는 클릭!
    driver.find_element(By.CSS_SELECTOR, '.qP6uz.SbSKx').click()
    time.sleep(0.2)

    # 결과보기 버튼 클릭
    driver.find_element(By.CSS_SELECTOR, '#modal-root .GZe5x.Mj_l3').click()
    time.sleep(0.5)


    df = pd.DataFrame();
    page = 0
    while True :

        # 무한 스크롤
        driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container').click()

        # 로딩된 데이터 개수 확인
        lis = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS.rTjJo')
        prev_len = len(lis)
        print(prev_len)
        
        while True :
            # 맨 아래로 스크롤 내림
            driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.END)    
            time.sleep(0.5)

            # 스크롤 내린후 li 갯수
            lis = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS.rTjJo')
            curr_len = len(lis)

            print(f'스크롤전:{prev_len}, 스크롤후:{curr_len}')
            if curr_len == prev_len :
                break
            prev_len = curr_len   

        # '광고'라는 단어를 포함하지 않는 'li' 요소들의 리스트를 생성합니다.
        lis = [li for li in lis if '광고' not in li.text]    

        print(f'광고제거후:{len(lis)}')
        
        # 한페이지의 리뷰가져오기
        tmp_df = get_reviews_of_stores (driver, lis[0:2], moreview=2)
        df = pd.concat([df,tmp_df])
        
        
        driver.
        
        next_page = driver.find_element(By.CSS_SELECTOR, 'div.zRM9F>:last-child')
        if next_page.get_attribute('aria-disabled') == 'false' :
            next_page.click()
            time.sleep(1)
            lis = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS.rTjJo')
            page += 1
            if page == 2:
                break
        else :
            break
          
            
    df.to_csv(f'{keywords}.csv', index=False)            

10
스크롤전:10, 스크롤후:10
광고제거후:8
10
스크롤전:10, 스크롤후:0
스크롤전:0, 스크롤후:0
광고제거후:0


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.zRM9F>:last-child"}
  (Session info: chrome=123.0.6312.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00DD4CA3+225091]
	(No symbol) [0x00D04DF1]
	(No symbol) [0x00BA9A7A]
	(No symbol) [0x00BE175B]
	(No symbol) [0x00BE188B]
	(No symbol) [0x00C17882]
	(No symbol) [0x00BFF5A4]
	(No symbol) [0x00C15CB0]
	(No symbol) [0x00BFF2F6]
	(No symbol) [0x00BD79B9]
	(No symbol) [0x00BD879D]
	sqlite3_dbdata_init [0x01249A43+4064547]
	sqlite3_dbdata_init [0x0125104A+4094762]
	sqlite3_dbdata_init [0x0124B948+4072488]
	sqlite3_dbdata_init [0x00F4C9A9+930953]
	(No symbol) [0x00D107C4]
	(No symbol) [0x00D0ACE8]
	(No symbol) [0x00D0AE11]
	(No symbol) [0x00CFCA80]
	BaseThreadInitThunk [0x76B6FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77BD7C5E+286]
	RtlGe

In [690]:
df

""


In [135]:
# 브라우져 세션 종료
driver.quit()